In [ ]:
import torch

import pandas as pd
import numpy as np

from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/Dacon/2/data/open.zip'

In [ ]:
train_df = pd.read_csv('./train.csv')
val_df = pd.read_csv('./val.csv')

In [ ]:
#fraud/normal 데이터로 anomaly_data의 비율 체크함
#train 데이터에서 ID를 지움으로서 차원축소

In [ ]:
val_normal, val_fraud = val_df['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

train_x = train_df.drop(columns=['ID'])

In [ ]:
#EllipticEnvelope 모델을 사용하였고 parameter는 다음과 같이 맞추어주었다.
#해당모델은 정규분포를 이용하여 데이터 분포에 타원을 그린다. 그리고 타원에서 벗어날수록 outlier이다.

In [ ]:
model = EllipticEnvelope(support_fraction = 0.994, contamination = val_contamination, random_state = 42)
model.fit(train_x)

In [ ]:
#model.score_samples를 하여 해당 데이터에 대한 로그 밀도 모델을 평가한다.
#그 뒤 해당 데이터를 tensor 자료형으로 변환한다.
#변환된 데이터에서 오름차순으로 k개를 반환한다.
#val 데이터로 print를 하여 F1 score를 평가한다.

In [ ]:
def get_pred_label(model, x, k):
  prob = model.score_samples(x)
  prob = torch.tensor(prob, dtype = torch.float)
  topk_indices = torch.topk(prob, k = k, largest = False).indices

  pred = torch.zeros(len(x), dtype = torch.long)
  pred[topk_indices] = 1
  return pred.tolist(), prob.tolist()

val_x = val_df.drop(columns=['ID', 'Class']) 
val_y = val_df['Class'] 

val_pred, val_prob = get_pred_label(model, val_x, 29)
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))
tn, fp, fn, tp = confusion_matrix(val_y, val_pred).ravel()

In [ ]:
test_df = pd.read_csv('./test.csv')
test_df.head()
test_x = test_df.drop(columns=['ID'])

In [ ]:
test_pred, _ = get_pred_label(model, test_x, 318)
print('n_fraud : ', sum(test_pred))

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit.head()

In [ ]:
submit['Class'] = test_pred
submit.to_csv('/content/submit_EllipticEnvelope.csv', index=False)

In [ ]:
pd.read_csv('/content/submit_EllipticEnvelope.csv')['Class'].sum()